In [1]:
!pip install -q transformers datasets librosa soundfile scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 45.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 21.0.0 which is incompatible.
bigframes 2.12.0 requires google-cloud-bigquery[bqstorage,pandas]>=3.31.0, but you have google-cloud-bigquery 3.25.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.1.0 which is incompatible.
gradio 5.38.1 requires pydantic<2.12,>=2.0, but you have pydantic

In [2]:
import os
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf")

In [3]:
from datasets import load_dataset

# Load dataset
dataset = load_dataset("NightPrince/MasriSpeech-Full")
print(dataset)

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

data/train-00000-of-00023.parquet:   0%|          | 0.00/437M [00:00<?, ?B/s]

data/train-00001-of-00023.parquet:   0%|          | 0.00/431M [00:00<?, ?B/s]

...

data/validation-00000-of-00001.parquet:   0%|          | 0.00/343M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50715 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2199 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 50715
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2199
    })
})


In [4]:
import pandas as pd
from datasets import Value

# Load translations CSV
csv_path = "/kaggle/input/translated/Translate - 1 (3).csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")
raw = df.iloc[:, 1].tolist()

# Normalize to str-or-None
def to_str_or_none(x):
    if x is None: 
        return None
    if isinstance(x, float) and pd.isna(x): 
        return None
    s = str(x).strip()
    return None if s == "" or s.lower() in {"nan", "null", "none"} else s

translations = [to_str_or_none(x) for x in raw]

# Pad/truncate to match dataset size
n_train = len(dataset["train"])
if len(translations) < n_train:
    translations += [None] * (n_train - len(translations))
else:
    translations = translations[:n_train]

# Define features with explicit string type for translation
new_feats = dataset["train"].features.copy()
new_feats["translation"] = Value("string")

# Add translation column using map with indices
def add_translation(batch, indices):
    return {"translation": [translations[i] for i in indices]}

dataset["train"] = dataset["train"].map(
    add_translation,
    with_indices=True,
    batched=True,
    features=new_feats,
    desc="Adding translation column"
)

# Verify
print(dataset)

Adding translation column:   0%|          | 0/50715 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'transcription', 'translation'],
        num_rows: 50715
    })
    validation: Dataset({
        features: ['audio', 'transcription'],
        num_rows: 2199
    })
})


In [5]:
# Check a few samples
chk = dataset["train"].select(range(3)).remove_columns(["audio"])
for i in range(3):
    trans = chk[i]['translation']
    trans_preview = trans[:50] + "..." if trans and len(trans) > 50 else trans
    print(f"  [{i}] translation: {trans_preview}")

# Count missing translations
missing = sum(t is None for t in dataset["train"]["translation"])
print(f"\n  Missing translations: {missing} / {n_train}")

  [0] translation: It was a shame that she was treating my only broth...
  [1] translation: Surely God compensated them with something better....
  [2] translation: Like many countries we see today in the world, the...

  Missing translations: 1387 / 50715


In [6]:
from datasets import Audio

# Cast audio column to NOT decode - this will give us bytes instead
dataset["train"] = dataset["train"].cast_column("audio", Audio(decode=False))

print("Audio configured to return raw bytes")
print(f"  Translations still present: {'translation' in dataset['train'].features}")

# Test that we can get bytes
test = dataset["train"][0]
print(f"\nTest audio data:")
print(f"  Has bytes: {'bytes' in test['audio'] and test['audio']['bytes'] is not None}")
print(f"  Bytes size: {len(test['audio']['bytes']) if test['audio']['bytes'] else 0} bytes")

Audio configured to return raw bytes
  Translations still present: True

Test audio data:
  Has bytes: True
  Bytes size: 228204 bytes


In [7]:
import os

# Remove old clone if exists
if os.path.exists("Benchmark-Whisper"):
    !rm -rf Benchmark-Whisper

# Clone repository
!git clone https://github.com/omar-A-hassan/Benchmark-Whisper.git

# Verify
if os.path.exists("Benchmark-Whisper/speech-to-text-benchmark"):
    print("Repository cloned successfully!")
    !ls -lh Benchmark-Whisper/speech-to-text-benchmark/*.py
else:
    print("Clone failed!")

Cloning into 'Benchmark-Whisper'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 91 (delta 21), reused 73 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 584.11 KiB | 11.45 MiB/s, done.
Resolving deltas: 100% (21/21), done.
Repository cloned successfully!
-rw-r--r-- 1 root root  11K Oct 12 15:03 Benchmark-Whisper/speech-to-text-benchmark/benchmark_latency.py
-rw-r--r-- 1 root root  11K Oct 12 15:03 Benchmark-Whisper/speech-to-text-benchmark/benchmark.py
-rw-r--r-- 1 root root  23K Oct 12 15:03 Benchmark-Whisper/speech-to-text-benchmark/dataset.py
-rw-r--r-- 1 root root  43K Oct 12 15:03 Benchmark-Whisper/speech-to-text-benchmark/engine.py
-rw-r--r-- 1 root root 6.2K Oct 12 15:03 Benchmark-Whisper/speech-to-text-benchmark/export_masrispeech_kaggle.py
-rw-r--r-- 1 root root  474 Oct 12 15:03 Benchmark-Whisper/speech-to-text-benchmark/languages.py
-rw-r--r-- 1 root ro

In [8]:
!pip install -r Benchmark-Whisper/speech-to-text-benchmark/requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.6/358.6 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 22.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.5/274.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.0/32.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [9]:
import sys

# Add the framework to Python path
framework_path = "/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark"
if framework_path not in sys.path:
    sys.path.insert(0, framework_path)

# Test imports
try:
    from languages import Languages
    from dataset import Datasets
    from engine import Engines
    from metric import Metrics
    
    print("Framework imports successful!")
    
    # Verify Arabic support
    print("\nVerifying Arabic support:")
    print(f"Arabic language: {Languages.AR}")
    print(f"MasriSpeech dataset: {Datasets.MASRI_SPEECH}")
    print(f"MasriSpeech translate: {Datasets.MASRI_SPEECH_TRANSLATE}")
    print(f"BLEU metric: {Metrics.BLEU}")
    
    # List available Whisper engines
    print("\nAvailable Whisper transcription engines:")
    for engine in [Engines.WHISPER_SMALL, Engines.WHISPER_BASE, Engines.WHISPER_SMALL, 
                   Engines.WHISPER_MEDIUM, Engines.WHISPER_LARGE_V3]:
        print(f"  • {engine.value}")
    
    print("\nAvailable Whisper translation engines:")
    for engine in [Engines.WHISPER_SMALL_TRANSLATE, Engines.WHISPER_BASE_TRANSLATE, 
                   Engines.WHISPER_SMALL_TRANSLATE, Engines.WHISPER_MEDIUM_TRANSLATE, 
                   Engines.WHISPER_LARGE_V3_TRANSLATE]:
        print(f"  • {engine.value}")
    
except Exception as e:
    print(f"Import failed: {e}")
    import traceback
    traceback.print_exc()

Import failed: No module named 'azure'


Traceback (most recent call last):
  File "/tmp/ipykernel_19/2487905008.py", line 12, in <cell line: 0>
    from engine import Engines
  File "/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/engine.py", line 18, in <module>
    import azure.cognitiveservices.speech as speechsdk
ModuleNotFoundError: No module named 'azure'


In [10]:
from export_masrispeech_kaggle import export_masrispeech_to_csv

# Export configuration
OUTPUT_FOLDER = "./masrispeech_benchmark"
AUDIO_FOLDER = "./masrispeech_benchmark/audio"

metadata_path, count = export_masrispeech_to_csv(
    dataset=dataset,
    output_folder=OUTPUT_FOLDER,
    audio_folder=AUDIO_FOLDER,
    split="train",
    max_samples=1000  # Change to None for all samples
)

print(f"\n Export complete: {count} samples")


Exporting MasriSpeech train split
  Output folder: ./masrispeech_benchmark
  Audio folder: ./masrispeech_benchmark/audio
  Total samples in split: 50715
  Exporting: 1000
  Has translations: True


Exporting train:   0%|          | 0/1000 [00:00<?, ?it/s]


Export Summary:
Exported: 1000 samples
  Metadata: ./masrispeech_benchmark/metadata.csv
  Audio files: ./masrispeech_benchmark/audio

 Export complete: 1000 samples


In [11]:
import pandas as pd

df = pd.read_csv(metadata_path)
print(f"Exported {len(df)} samples\n")
print(df.head(3))

# Check translations
if 'translation' in df.columns:
    with_trans = (df['translation'].notna() & (df['translation'] != "")).sum()
    print(f"\n Samples with translation: {with_trans}/{len(df)}")

Exported 1000 samples

                           audio_path  \
0  audio/masrispeech_train_00000.flac   
1  audio/masrispeech_train_00001.flac   
2  audio/masrispeech_train_00002.flac   

                                       transcription  \
0  على إنها عار في الوقت اللي كانت بتتعامل مع أخو...   
1  فأكيد ربنا عوضهم خير هو الراجل بيبقى ليه إختيا...   
2  زي دول كتيره بنشوفها النهارده في العالم وأصبحت...   

                                         translation  
0  It was a shame that she was treating my only b...  
1  Surely God compensated them with something bet...  
2  Like many countries we see today in the world,...  

 Samples with translation: 1000/1000


In [12]:
os.chdir("/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark")


In [13]:
import os

os.chdir("/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark")

# Clean up before running
print("Cleaning up cache and old results...")
!rm -f /kaggle/working/masrispeech_benchmark/audio/*.wspt  # Delete transcription cache
!rm -f results/AR/MASRI_SPEECH/WHISPER_SMALL.log  # Delete old results

# Run benchmark
!python3 benchmark.py \
    --engine WHISPER_SMALL \
    --dataset MASRI_SPEECH \
    --dataset-folder /kaggle/working/masrispeech_benchmark \
    --language AR \
    --num-examples 1000 \
    --num-workers 1

os.chdir("/kaggle/working")

# Display results
print("\n Transcription benchmark complete!")
print("\n=== Results ===")
!cat /kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/results/AR/MASRI_SPEECH/WHISPER_SMALL.log

Cleaning up cache and old results...
Traceback (most recent call last):
  File "/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/benchmark.py", line 17, in <module>
    from engine import (
  File "/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/engine.py", line 18, in <module>
    import azure.cognitiveservices.speech as speechsdk
ModuleNotFoundError: No module named 'azure'

 Transcription benchmark complete!

=== Results ===
cat: /kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/results/AR/MASRI_SPEECH/WHISPER_SMALL.log: No such file or directory


In [14]:
import os

os.chdir("/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark")

# Clean up before running
print("Cleaning up cache and old results...")
!rm -f /kaggle/working/masrispeech_benchmark/audio/*.wspt_trans  # Delete translation cache
!rm -f results/AR/MASRI_SPEECH_TRANSLATE/WHISPER_SMALL_TRANSLATE.log  # Delete old results

# Run benchmark
!python3 benchmark.py \
    --engine WHISPER_SMALL_TRANSLATE \
    --dataset MASRI_SPEECH_TRANSLATE \
    --dataset-folder /kaggle/working/masrispeech_benchmark \
    --language AR \
    --num-examples 1000 \
    --num-workers 1

os.chdir("/kaggle/working")

# Display results
print("\n Translation benchmark complete!")
print("\n=== Results ===")
!cat /kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/results/AR/MASRI_SPEECH_TRANSLATE/WHISPER_SMALL_TRANSLATE.log

Cleaning up cache and old results...
Traceback (most recent call last):
  File "/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/benchmark.py", line 17, in <module>
    from engine import (
  File "/kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/engine.py", line 18, in <module>
    import azure.cognitiveservices.speech as speechsdk
ModuleNotFoundError: No module named 'azure'

 Translation benchmark complete!

=== Results ===
cat: /kaggle/working/Benchmark-Whisper/speech-to-text-benchmark/results/AR/MASRI_SPEECH_TRANSLATE/WHISPER_SMALL_TRANSLATE.log: No such file or directory
